# **Predicting Concrete Compressive Strength**

### Motivation:
Concrete is a fundamental material in civil engineering. Its compressive strength is important for ensuring the safety and durability of structures. However, determining this strength accurately is challenging because it depends on a nonlinear function between several factors, including the age of the concrete and the proportions of its components—such as cement, blast furnace slag, fly ash, water, superplasticizer, coarse aggregate, and fine aggregate.
This project aims to apply machine learning techniques to improve the prediction of concrete strength, in order to contribute to better design, quality control, and resource optimization in construction.

### Dataset Explanation:
The dataset is sourced from the UCI Machine Learning Repository and contains 1030 observations with 8 quantitative input variables, and 1 quantitative output variable.
The input variables are: cement, blast furnace slag, fly ash, water, superplasticizer, coarse aggregate, fine aggregate and age. Except for age, the other variables are ingredient quantities in the concrete mixture and measured in *kg in a $m^{3}$* mixture. The age is measured in *days*.
The target, concrete compressive strengt, is measured in *MPa*.

### Data Analytics: Preprocessing
Since there is no missing attribute values according to the information provided by UCI Machine Learning Repository. Data can be directly read from Concrete_data.csv.

In [ ]:
import statsmodels.api as sm
import numpy as np
import pandas as pd
import seaborn as sns
import scipy
import sklearn

from matplotlib import pyplot as plt
from pandas.plotting import scatter_matrix
from itertools import combinations

from statsmodels.stats.outliers_influence import OLSInfluence
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.model_selection import KFold, cross_val_score, train_test_split, GridSearchCV
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error, make_scorer, r2_score

from sklearn.model_selection import KFold, cross_validate

%matplotlib inline
plt.style.use('ggplot')
sns.set_style("darkgrid")
# print numpy arrays with precision 4
np.set_printoptions(precision=4)

In [ ]:
concrete_df = pd.read_csv('./Concrete_Data.csv')
concrete_df.columns = concrete_df.columns.str.strip()
concrete_df.head()

In [ ]:
#define all the variables we need for the linear regression with concrete compressive strength as response and all other variables as possible features
label_column = 'concrete_compressive_strength'
feature_columns = [c for c in concrete_df.columns if c != label_column]
X = concrete_df[feature_columns].values
y = concrete_df[label_column].values
p = len(feature_columns)
print("Features are:", feature_columns,
      ", and the response variable is:", label_column)

#### Explanation of relevant variables:

The following table shows the meaning and expected effect of respective variables on strength.
| Variable             | Meaning                              | Expected Effect on Strength    |
| -------------------- | ------------------------------------ | ------------------------------ |
| `cement`             | Binder                               | increases strength          |
| `blast_furnace_slag` | Supplementary binder                 | increases strength or neutral                  |
| `fly_ash`            | Supplementary binder                 | increase strength or neutral                  |
| `water`              | Affects workability and hydration    | decreases strength (excess) |
| `superplasticizer`   | Increases workability, reduces water | increase or decrease depending on context  |
| `coarse_aggregate`   | Filler                               | neutral or mild effect         |
| `fine_aggregate`     | Filler                               | neutral or mild effect         |
| `age`                | Curing time                          | increases strength          |


#### Correlation between relevant variables:

In [ ]:
corr = concrete_df.corr(numeric_only=True)
sns.heatmap(corr, annot=True, cmap="coolwarm")
plt.show()

#### Variables that influence concrete strength:

According to the result of the heatmap, it can be seen that **cement** has a moderately strong positive correlation with concrete strength with a value of +0.50, which indicates more cement usually leads to stronger concrete. **Superplasticizer** also has a positive effect with a value of +0.37.
**Age** has a positive effect too , because concrete gets stronger over time.
**Water** has a negative correlation with strength of -0.29, which means more water tends to weaken concrete. This observation is consistent with **Abrams' water-to-cement ratio pronouncement**, which emphasizes that increased water relative to cement typically reduces concrete strength.

#### Multicollinearity:

The highest multicollinearity is the correlation between **water and superplasticizer**, which has a value of -0.66. This indicates they carry overlapping and opposite information. There is also moderate correlation among **cement**, **blast furnace slag**, and **fly ash** because these may act as partial substitutes in the concrete mix, so they tend to move in opposite directions.

In [ ]:
VIFs = [(predictor, variance_inflation_factor(X,_)) \
        for _,predictor in enumerate(list(feature_columns))]
print('Variance Inflation Factors')
for tup in VIFs:
    print('{:20}'.format(tup[0]), '{:.3f}'.format(tup[1]))

The variance inflation factors are larger for cement, blast furnace slag, fly ash, water, coarse aggregate, fine aggregate. This also indicates multi-collinearity of the factors mention above.

As mentioned in the lecture content of **MLDA**, VIF equal to 1 shows the absence of collinearity Usually, there is collinearity when VIF is larger than 5.

### Machine Learning Methods:
This project aims to predict concrete compressive strength which is a problem with a quantitative response. According to the lecture content in the fundamental part of Machine Learning and Data Analytics(MLDA), this task is a **regression task**. In the following part, linear regression, ridge regression, lasso regression, one decision-tree based method are applied.

### Cross validation:

#### K-fold cross validation:

For supervised machine learning, the goal is to develop a model that can perform accurately not only on the data it was trained on, but also on previously unseen data. To evaluate a model's generalization ability, the **K-fold** method is utilized as the cross validation method.

In [ ]:
# define the kfold, shuffle the training and test dataset every time
kf = KFold(n_splits=10, random_state = 0, shuffle = True)

##### Scores for the model

Mean squared error:

In [ ]:
# Gets the MSE for a model with zero features (just predicting the mean)
def get_base_mse(y):
    predictions = [np.mean(y)] * len(y)
    return mean_squared_error(predictions, y)

# Gets the MSE of our model, for a given dataset, estimated with 10-fold cross-validation.
def k_fold_mse(X, y):
    scores_mse = cross_validate(model, X, y, scoring = 'neg_mean_squared_error', cv = kfold)
    return np.mean(scores['test_score']) * -1

$R^2$:

#### Linear Regression:

##### Statsmodel, outliers and high leverege points:

In [ ]:
#Statsmodel fitting, it is used to show the p-values
X_stat = sm.add_constant(concrete_df.iloc[:,1:])
y_stat = concrete_df.concrete_compressive_strength

statmodel = sm.OLS(y_stat,X_stat)
estimate = statmodel.fit()

print(estimate.summary())

According to the result of the fitted model, there is a relationship between the response of concrete compressive strength and **cement, blast furnace slag, fly ash, water, superplasticizer and age**, because they have low p-values, which is smaller than 0.05.

In [ ]:
# Obtain the residuals, studentized residuals and the leverages
fitted_values = estimate.fittedvalues
residuals = estimate.resid.values
studentized_residuals = OLSInfluence(estimate).resid_studentized_internal
leverages = OLSInfluence(estimate).influence

In [ ]:
# Calculate thresholds
n = len(fitted_values)
p = X.shape[1] - 1  # exclude constant
leverage_thresh = (p + 1) / n

# Convert to arrays
studentized_residuals = np.asarray(studentized_residuals)
leverages = np.asarray(leverages)

# Identify outlier indices
outlier_indices = np.where(np.abs(studentized_residuals) > 3)[0]
high_leverage_indices = np.where(leverages > leverage_thresh)[0]
outliers = []
for idx in outlier_indices:
    outliers.append(idx)
print("Outliers are:",outliers)

# Find common indices (both outlier and high leverage)
joint_outliers = np.intersect1d(outlier_indices, high_leverage_indices)
high_leverage_and_outliers = []

for idx in joint_outliers:
    high_leverage_and_outliers.append(idx)
print("Comments on joint outliers (in both ax2 and ax3):",high_leverage_and_outliers)


# Plot
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(16, 4))

# 1. Residuals plot
ax1.scatter(fitted_values, residuals, facecolors='none', edgecolors='b')
ax1.set_xlabel('Fitted values')
ax1.set_ylabel('Residuals')

# 2. Studentized Residuals plot
ax2.scatter(fitted_values, studentized_residuals, facecolors='none', edgecolors='b')
ax2.axhline(y=3, color='r', linestyle='--', linewidth=1)
ax2.axhline(y=-3, color='r', linestyle='--', linewidth=1)
# Mark joint outliers
for idx in joint_outliers:
    ax2.scatter(fitted_values[idx], studentized_residuals[idx], color='red')
    ax2.annotate(idx, (fitted_values[idx], studentized_residuals[idx]), color='red', fontsize=8)
ax2.set_xlabel('Fitted values')
ax2.set_ylabel('Studentized residuals')

# 3. Leverage vs Studentized Residuals
ax3.scatter(leverages, studentized_residuals, facecolors='none', edgecolors='b')
ax3.axhline(y=3, color='r', linestyle='--', linewidth=1)
ax3.axhline(y=-3, color='r', linestyle='--', linewidth=1)
ax3.axvline(x=leverage_thresh, color='g', linestyle='--', linewidth=1)
# Mark joint outliers
for idx in joint_outliers:
    ax3.scatter(leverages[idx], studentized_residuals[idx], color='red')
    ax3.annotate(idx, (leverages[idx], studentized_residuals[idx]), color='red', fontsize=8)
ax3.set_xlabel('Leverage')
ax3.set_ylabel('Studentized residuals')

plt.tight_layout()
plt.show()

The residuals display a clear pattern, which suggests **non-linearity** in the data. The studentized residuals indicate the presence of two outliers. The leverage plot also reveals several high-leverage points—defined as points with leverage greater than $(p+1)/n=9/1030\approx 0.009$. Notably, both outliers are also high-leverage points. However, since the dataset does not clearly define criteria for excluding data points, all observations, including the outliers, are retained for the subsequent analysis.

##### Scikit-Model:

In [ ]:
# Scikit-Model
model = LinearRegression()
mse_per_fold_lin = []

for i, (train_idx, test_idx) in enumerate(kf.split(X)):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    mse = mean_squared_error(y_test, preds)
    mse_per_fold_lin.append(mse)

    # Print fold summary: remove the"#"
    # print(f"\n--- Fold {i+1} ---")
    # print(f"MSE: {mse:.4f}")
    # print(f"Intercept: {model.intercept_:.4f}")
    # print(f"Coefficients: {model.coef_}")
print("MSE for each fold:",mse_per_fold_lin)
print("\nAverage MSE across folds:", np.mean(mse_per_fold_lin))

#### Polynomial Regression:

Since there is non-linearity in the data, we use the polynomial regression to train the dataset.

In [ ]:
# Create MSE scorer for polynomial regression
mse_scorer = make_scorer(mean_squared_error)

# Store results
results = {}
for feature in feature_columns:

    #Store MSE
    mse_poly = []

    for degree in range(1, 6):  # Degrees 1 to 5
            # Create polynomial features
            poly = PolynomialFeatures(degree=degree)
            X_poly = poly.fit_transform(X)

            # Create linear regression model
            model = LinearRegression()

            # Perform 10-fold CV and get negative MSE (sklearn returns negative MSE by default)
            mse_scores = cross_val_score(model, X_poly, y, scoring=mse_scorer, cv=10)

            # Store average MSE
            mse_poly.append(np.mean(mse_scores))

        # Store MSEs for this feature
    results[feature] = mse_poly
    # Plot results

plt.figure(figsize=(12, 6))

for feature, mse_list in results.items():
    plt.plot(range(1, 6), mse_list, marker='o', label=feature)

    # Sort MSEs with their degree indices
    sorted_mse = sorted((mse, i+1) for i, mse in enumerate(mse_list))  # (MSE, degree)

    min_mse, best_degree = sorted_mse[0]
    second_min_mse, second_best_degree = sorted_mse[1]

    print(f"{feature}:")
    print(f"  Lowest MSE = {min_mse:.2f} at degree {best_degree}")
    print(f"  Second Lowest MSE = {second_min_mse:.2f} at degree {second_best_degree}")

plt.xlabel('Polynomial Degree')
plt.ylabel('Mean Squared Error')
plt.title('Polynomial Regression MSE (Each Feature Individually)')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()


$\text{concrete\_compressive\_strength} = \beta_0
+ \beta_1 \cdot \text{cement}
+ \beta_2 \cdot \text{blast\_furnace\_slag}^5
+ \beta_3 \cdot \text{fly\_ash}^4
+ \beta_4 \cdot \text{water}^4
+ \beta_5 \cdot \text{superplasticizer}
+ \beta_6 \cdot \text{coarse\_aggregate}^2
+ \beta_7 \cdot \text{fine\_aggregate}
+ \beta_8 \cdot \text{age}^4
$

#### Polynomial Degree Selection Summary and Justification:

In order to model the relationship between the features and the concrete compressive strength, polynomial regression up to degree 5 is applied for each feature. The mean squared error (MSE) was computed using 10-fold cross-validation to assess the predictive performance at each polynomial degree.

While some features achieved their lowest MSE at higher degrees, the improvements over lower degrees were often marginal. Since high-degree polynomial models tend to overfit—capturing noise instead of the underlying patter. Both the lowest and second-lowest MSEs are considered for each feature to guide model complexity choices.

Below is a summary of the best and second-best MSEs and their associated degrees:

| Feature            | Lowest MSE | Degree | Second Lowest MSE | Degree |
| ------------------ | ---------- | ------ | ----------------- | ------ |
| Cement             | 232.98     | 1      | 235.47            | 2      |
| Blast Furnace Slag | 303.89     | 5      | 304.91            | 2      |
| Fly Ash            | 317.01     | 4      | 317.38            | 1      |
| Water              | 248.95     | 4      | 259.49            | 3      |
| Superplasticizer   | 272.21     | 1      | 277.66            | 2      |
| Coarse Aggregate   | 296.42     | 2      | 296.86            | 1      |
| Fine Aggregate     | 293.30     | 1      | 295.24            | 2      |
| Age                | 206.15     | 4      | 211.45            | 5      |


In features such as blast furnace slag and fly ash, the difference in MSE between the best and second-best degrees is negligible (≈1 unit). In such cases, lower-degree polynomials were selected to avoid overfitting while maintaining nearly optimal performance. For example, blast furnace slag was assigned degree 2 instead of 5 due to only a 1.02 unit difference in MSE.
Consequently, degree 4 was the highest degree ultimately selected across all features. Therefore, the maximum polynomial degree for the multivariate polynomial regression model was set to 4. This approach ensures a balance between model flexibility and generalization.

#### Ridge regression:

In [ ]:
label_column = 'concrete_compressive_strength'
feature_columns = ['cement', 'blast_furnace_slag', 'fly_ash', 'water', 'superplasticizer', 'coarse_aggregate', 'fine_aggregate', 'age']
X_raw = concrete_df[feature_columns].values
y_raw = concrete_df[label_column].values
p = len(feature_columns)


Since degree 4 was the highest optimal degree observed across all features after this adjustment, we selected degree 4 as the maximum polynomial degree

In [ ]:
# define the model
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
model = make_pipeline(
    StandardScaler(),
    PolynomialFeatures(degree=2),
    GridSearchCV(
        estimator = Ridge(),
        cv = kfold,
        scoring = 'neg_mean_squared_error',

        # param_grid determines the parameters to test (alpha is lambda in the Ridge estimator)
        # np.logspace(-3, 2, 50): array from 10^-3 to 10^2 in 50 steps (base default is 10, can also be something else)
        param_grid = {'alpha': np.logspace(-3, 2, 50)},
    )
)
model.fit(X_raw, y_raw)

# obtain the results
lambdas = [p['alpha'] for p in model[2].cv_results_['params']]
mses = [neg_mse * -1 for neg_mse in model[2].cv_results_['mean_test_score']]
best_model = model.named_steps['gridsearchcv'].best_estimator_
best_mse = min(mses)
best_lambda = lambdas[np.argmin(mses)]

print(f"Best alpha:{best_lambda}:with MSE: {best_mse:.2f}")
model

In [ ]:
# plot the reults
fig, ax = plt.subplots(figsize = (12, 8))
ax.plot(lambdas, mses)
ax.set_xscale('log')
ax.set_title(r"MSE as a function of $\lambda$")
ax.set_xlabel(r"Hyper-parameter $\lambda$")
ax.set_ylabel("Estimated MSE");
plt.show()

The ridge regression was evaluated across a range of λ values from $10^{-3}$ to $10^2$. The validation MSE decreased with small increases in λ due to reduced overfitting, reaching a minimum at λ = 4.71. Beyond this point, further increases in λ caused the model to underfit the data, resulting in higher MSEs.
Thus, the optimal λ value was selected as 4.71, where the model achieved the lowest MSE of 34.56.

In [ ]:
lambdas = np.logspace(-3, 6, 50)
coefficients = list()

# for each lambda define and fit the model and save the obtained parameters in the coefficients list
for lam in lambdas:
    model = make_pipeline(
        StandardScaler(),
        PolynomialFeatures(degree = 2),
        Ridge(alpha = lam)
    )
    model.fit(X_raw, y_raw)
    coefficients.append(model[2].coef_)


# plot the results
coefficients = np.array(coefficients).T
fig, ax = plt.subplots(figsize = (12, 10))

for coef_vals in coefficients:
    ax.plot(lambdas, coef_vals)

ax.set_xlabel(r"Hyper-parameter $\lambda$")
ax.set_ylabel(r"Value of the model coefficients $\beta_i$")
ax.set_xscale('log');

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_raw, y_raw, test_size=0.2, random_state=42
)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
model = Ridge(alpha=best_lambda)
model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
print(f"Test MSE: {mse:.2f}")


In [ ]:
#show the model
feature_names = ['cement', 'blast_furnace_slag', 'fly_ash', 'water',
                 'superplasticizer', 'coarse_aggregate',
                 'fine_aggregate', 'age']
coefficients = model.coef_
intercept = model.intercept_
#show the equation
equation = f"concrete compressive strength = {intercept:.3f}"
for coef, name in zip(coefficients, feature_names):
    equation += f" + ({coef:.3f} * {name})"
print(equation)